<a href="https://colab.research.google.com/github/m-richa/Co-domain-adaptation-for-image-classification/blob/main/codomain_adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git remote add origin https://<USERNAME>:<PASSWORD>@github.com/<USERNAME>/reponame.git

In [2]:
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm
import tensorflow as tf
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Flatten, Input
import tensorflow_datasets as tfds
from tensorflow.python.keras.utils.vis_utils import plot_model
import tensorflow.keras.backend as K
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [6]:
#!unzip -q photo.zip
!unzip -q sketch.zip

In [4]:
IMG_SIZE = 224
BATCH_SIZE = 50
EPOCHS = 1
src_path_train = '/content/photo/'
trg_path = '/content/sketch/'

In [5]:
from tensorflow.keras.utils import Sequence
import matplotlib.pyplot as plt
import numpy as np
import os
import random


class MergedGenerators(Sequence):

    

    def __init__(self, batch_size, generators=[], sub_batch_size=[]):
        self.generators = generators
        self.sub_batch_size = sub_batch_size
        self.batch_size = batch_size

    def __len__(self):
        return int(
            sum([(len(self.generators[idx]) * self.sub_batch_size[idx])
                 for idx in range(len(self.sub_batch_size))]) /
            self.batch_size)

    def __getitem__(self, index):
        """Getting items from the generators and packing them"""

        X_batch = []
        Y_batch = []
        for generator in self.generators:
            if generator.class_mode is None:
                x1 = generator[index % len(generator)]
                X_batch = [*X_batch, *x1]

            else:
                x1, y1 = generator[index % len(generator)]
                X_batch = [*X_batch, *x1]
                Y_batch = [*Y_batch, *y1]

        if self.generators[0].class_mode is None:
            return np.array(X_batch)
        return np.array(X_batch), np.array(Y_batch)


def build_datagenerator(dir1=None, dir2=None, batch_size=32):


    seed = random.randint(0, 100)
    print("seed={}".format(seed))
    n_images_in_dir1 = sum([len(files) for r, d, files in os.walk(dir1)])
    n_images_in_dir2 = sum([len(files) for r, d, files in os.walk(dir2)])

    # Have to set different batch size for two generators as number of images
    # in those two directories are not same. As we have to equalize the image
    # share in the generators
    generator1_batch_size = int((n_images_in_dir1 * batch_size) /
                                (n_images_in_dir1 + n_images_in_dir2))

    generator2_batch_size = batch_size - generator1_batch_size

    generator1 = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=5.,
        horizontal_flip=True,
    )

    generator2 = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=5.,
        horizontal_flip=True,
    )

    # generator2 has different image augmentation attributes than generaor1
    generator1 = generator1.flow_from_directory(
        dir1,
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=generator1_batch_size,
        shuffle=True,
        seed=seed,
        interpolation="bicubic",
    )

    generator2 = generator2.flow_from_directory(
        dir2,
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='rgb',
        class_mode=None,
        batch_size=generator2_batch_size,
        shuffle=True,
        seed=seed,
        interpolation="bicubic",
    )

    return MergedGenerators(
        batch_size,
        generators=[generator1, generator2],
        sub_batch_size=[generator1_batch_size, generator2_batch_size])


def train_datagen(batch_size=BATCH_SIZE):
    datagen = build_datagenerator(dir1=src_path_train,
                                  dir2=trg_path,
                                  batch_size=batch_size)

    print("Datagenerator length (Batch count):", len(datagen))
    
    '''for batch_count, image_batch in enumerate(datagen):
        if batch_count == 1:
            break

        image_batch = np.array(image_batch)
        #print(image_batch[0][1].shape)

        #print("Images: ", image_batch.shape)
        
        plt.figure(figsize=(10, 10))
        for i in range(10):
            plt.subplot(1, batch_size, i + 1)
            plt.imshow(image_batch[0][i], interpolation='nearest')
            plt.axis('off')
            plt.tight_layout()'''

    return datagen




In [ ]:
train_data_gen.__getitem__(1)[0][:2].shape

In [15]:
class model(tf.keras.Model):

    def __init__(self, num_classes=7):
        super(model, self).__init__()

        self.input_layer = tf.keras.layers.Input(name="inp_img", shape=(224,224,3,))
        self.feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224,224,3,),
                                               include_top=False,
                                               weights='imagenet')
        self.feature_extractor.trainable=False
        self.l1 = tf.keras.layers.GlobalAveragePooling2D()
        self.l2 = tf.keras.layers.Flatten()
        self.l3 = tf.keras.layers.Dense(1024, activation="relu")
        self.l4 = tf.keras.layers.Dense(512, activation="relu")
        self.l5 =tf.keras.layers.Dense(7, activation="softmax", name="logits")

        #super(model, self).__init__(inputs=self.input_layer ,outputs=self.l5,)
    
    def summary(self):
        x = self.input_layer
        model = Model(inputs=[x], outputs=self.call(x))
        return model.summary()

    def call(self, inputs):
        x = self.feature_extractor(inputs)
        x.trainable = False

        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)

        return x

'''if __name__ == '__main__':
    model = model()
    model.summary()'''

"if __name__ == '__main__':\n    model = model()\n    model.summary()"

In [8]:
def weighted_cross_entropy_loss(logits):
    logit_max = K.max(logits, 1)
    y_true = K.argmax(logits)
    
    is_condition = logit_max>=0.95
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(y_true, logits)
    weighted_loss = tf.multiply(logit_max, loss)
    return tf.where(is_condition, weighted_loss, 0)

In [21]:
def loss_object(logits, labels):
  loss1 = tf.keras.losses.categorical_crossentropy(labels, logits[:14])
  loss2 = weighted_cross_entropy_loss(logits[14:])
  print(logits[14])
  loss3 = tf.reduce_mean([x*tf.math.log(x) for x in logits[14:]])

  return 4*loss1, 2*loss2, 0.0 #loss3

In [10]:
optimizer = tf.keras.optimizers.Adam()

In [11]:
BATCH_SIZE= 100
datagen = build_datagenerator(dir1=src_path_train,
                                  dir2=trg_path,
                                  batch_size=batch_size)

EPOCHS = len(datagen)

def train_steps(optimizer, model, step):


  train_data_gen = train_datagen(BATCH_SIZE)
  
  loss_1, loss_2, loss_3, loss_value = [], [], [], []
  #pbar1 = tqdm(total=112, position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')
  #pbar2 = tqdm(total=112, position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')
  #pbar3 = tqdm(total=112, position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')
  pbar = tqdm(total=112, position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')

  for i in range(len(train_data_gen)):
    img_src = train_data_gen.__getitem__(i)[0]#[:10]
    src_label = train_data_gen.__getitem__(i)[1]
    #print(train_data_gen.__getitem__(i)[0].shape)
    #print(train_data_gen.__getitem__(i)[1].shape)

    with tf.GradientTape() as tape:
      logits = model(img_src, training=True)
      loss_value1, loss_value2, loss_value3 = loss_object(logits, src_label)
      total_value = tf.reduce_mean(loss_value1) + tf.reduce_mean(loss_value2) + tf.reduce_mean(loss_value3)
  
    gradients = tape.gradient(total_value, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    loss_value.append(total_value)
    loss_1.append(tf.reduce_mean(loss_value1))
    loss_2.append(tf.reduce_mean(loss_value2))
    loss_3.append(tf.reduce_mean(loss_value3))
      
    #train_acc_metric(y_batch_train, logits)
    #pbar1.set_description("Training loss1 for step %s: %.4f" % (int(step), float(tf.reduce_mean(loss_1))))
    #pbar2.set_description("Training loss2 for step %s: %.4f" % (int(step), float(tf.reduce_mean(loss_2))))
    #pbar3.set_description("Training loss3 for step %s: %.4f" % (int(step), float(tf.reduce_mean(loss_3))))
    pbar.set_description("Training loss for step %s: %.4f" % (int(step), float(tf.reduce_mean(loss_value))))
    #pbar1.update()
    #pbar2.update()
    #pbar3.update()
    pbar.update()
  
  return loss_value, loss_1, loss_2, loss_3



In [14]:
del model

In [16]:
#del model
model = model()


In [22]:


# Iterate over epochs.
epochs = 20
epochs_train_losses, epochs_train_losses1, epochs_train_losses2, epochs_train_losses3 = [], [], [], []

for epoch in range(epochs):
  print('Start of epoch %d' % (epoch,))
  
  losses_train, loss_1, loss_2, loss_3 = train_steps(optimizer, model, epoch)

  losses_train_mean = np.mean(losses_train)
  losses_train_mean1 = np.mean(loss_1)
  losses_train_mean2 = np.mean(loss_2)
  losses_train_mean3 = np.mean(loss_3)

  epochs_train_losses.append(losses_train_mean)
  epochs_train_losses.append(losses_train_mean1)
  epochs_train_losses.append(losses_train_mean2)
  epochs_train_losses.append(losses_train_mean3)

  print('\n Epoch %s: Train loss: %.4f  Loss1: %.4f, Loss2: %.4f, Loss3: %.4f' % (epoch, float(losses_train_mean), float(losses_train_mean1), float(losses_train_mean2), float(losses_train_mean3)))
  


Start of epoch 0
seed=46
Found 1670 images belonging to 7 classes.
Found 3929 images belonging to 7 classes.


  0%|          | 0/112 

Datagenerator length (Batch count): 112


Training loss for step 0: 8.2033: 100%|██████████| 112/112 


 Epoch 0: Train loss: 8.2033  Loss1: 8.1792, Loss2: 0.0241, Loss30.0000
Start of epoch 1
seed=74
Found 1670 images belonging to 7 classes.
Found 3929 images belonging to 7 classes.


...


 Epoch 18: Train loss: 5.4059  Loss1: 5.4059, Loss2: 0.0000, Loss30.0000
Start of epoch 19
seed=88
Found 1670 images belonging to 7 classes.
Found 3929 images belonging to 7 classes.


  0%|          | 0/112 

Datagenerator length (Batch count): 112


Training loss for step 19: 5.5059: 100%|██████████| 112/112 


 Epoch 19: Train loss: 5.5059  Loss1: 5.5059, Loss2: 0.0000, Loss30.0000
